<a href="https://www.kaggle.com/code/bugsydor/cleaning-water-data?scriptVersionId=142920431" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

The goal here is to take groundwater data across three years, combine them into a single dataframe, and clean it all enough to be used in a hypothetical analysis project.

In [ ]:
# Read in the three CSVs
water_2018 = pd.read_csv("/kaggle/input/telangana-post-monsoon-ground-water-quality-data/ground_water_quality_2018_post.csv") # 26 columns
## Replacing the season column in each frame with a year column, since it'll be easier to interpret as a year and I don't lose any info
water_2018.drop('season', axis = 1, inplace = True)
water_2018['year'] = 2018
#water_2018.head()

water_2019 = pd.read_csv("/kaggle/input/telangana-post-monsoon-ground-water-quality-data/ground_water_quality_2019_post.csv") # 26 columns
# Replacing the season column in each frame with a year column, since it'll be easier to interpret as a year and I don't lose any info
water_2019.drop('season', axis = 1, inplace = True)
water_2019['year'] = 2019
#water_2019.head()

water_2020 = pd.read_csv("/kaggle/input/telangana-post-monsoon-ground-water-quality-data/ground_water_quality_2020_post.csv") # 27 columns, let's fix that by dropping the empty one
water_2020.drop('Unnamed: 8', axis = 1, inplace = True)
# Replacing the season column in each frame with a year column, since it'll be easier to interpret as a year and I don't lose any info
water_2020.drop('season', axis = 1, inplace = True)
water_2020['year'] = 2020
water_2020.head()
water_2020.head()

In [ ]:
water_2018.dtypes

In [ ]:
water_2019.convert_dtypes().dtypes

In [ ]:
water_2019.iloc[:, 11:14]

In [ ]:
water_2020.dtypes

It seems they were not consistent with their column labels across all three years, or with the dtypes of the columns. Some Ints will need converting to floats, and some ions will need simplifying to fall in line.

In [ ]:
# find bad rows in 2020
bad_rows = water_2020.loc[water_2020.pH.str.contains('..', regex = False)]
bad_rows # There was exactly one bad row: sno 269 had a pH of 8..05

# repair the one bad cell in pH
water_2020.at[261, 'pH'] = 8.05

In [ ]:
# correct dtypes in 2018
water_2018[['CO3', 'Cl', 'Ca']] = water_2018[['CO3', 'Cl', 'Ca']].astype('float64')

# correct dtypes in 2019
water_2019.iloc[:, 11:14] = water_2019.iloc[:, 11:14].astype('float64')

# correct dtypes in 2020
water_2020[['pH', 'HCO3', 'Cl']] = water_2020[['pH', 'HCO3', 'Cl']].astype('float64')

water_2019.dtypes
#water_2020.dtypes

In [ ]:
# correct 2019 column names to line up with the other years'
water_2019.columns.values[8]  = 'E.C'
water_2019.columns.values[10] = 'CO3'
water_2019.columns.values[11] = 'HCO3'
water_2019.columns.values[12] = 'Cl'
water_2019.columns.values[13] = 'F'
water_2019.columns.values[14] = 'NO3 '
water_2019.columns.values[15] = 'SO4'
water_2019.columns.values[16] = 'Na'
water_2019.columns.values[17] = 'K'
water_2019.columns.values[18] = 'Ca'
water_2019.columns.values[19] = 'Mg'

#water_2019.dtypes # All column names should line up across the years now.

In [ ]:
# Check each table for columns that have NAs
water_2018.isnull().any() # 'gwl' contains NAs
water_2019.isnull().any() # 'gwl' and 'CO3' contain NAs
water_2020.isnull().any() # 'gwl' contains NAs

CO3 is the reading for dissolved carbonates, and is potentially important. 44% of its values in 2019 are missing. I doubt that I can effectively use the carbonate data for anything including 2019 with this much of it missing, so the column should probably be dropped for any multi-year analysis. Still, it may be possible to approximate the missing carbonate levels off of the sum of the Ca and Mg ion readings, since it seems to be the [most common](https://www3.uwsp.edu/cnr-ap/weal/Documents/HOPinterp.pdf) anion complement to those cations in untreated groundwater.

GWL is the ground water level, and is likely important. Around 1% of its values across all years are missing. The entries missing it could likely be ignored, or their values imputed, with little impact.

In [ ]:
# Combining all three years into a single frame
water_all = pd.concat([water_2018, water_2019, water_2020], axis = 0)
water_all.sample(5)

In [ ]:
# check for nonsense values where applicable

## pH, lat_gis, long_gis
water_all[['pH', 'lat_gis', 'long_gis']].describe()
    ### Latitude and longitude don't appear to have any egregious errors/typos, and are all believable values with no outliers.
    ### pH all falls within believable values as well, though the max being just shy of 10.5 is a bit concerning for the people drinking it.

## classification levels
water_all['Classification'].value_counts() # contains 2 O.G that should be labeled as OG
#water_all['Classification.1'].value_counts() # no typos here


In [ ]:
# fix the O.G to OG
water_all.loc[water_all.Classification == 'O.G', 'Classification'] = 'OG'

In [ ]:
# get a slice of the concentration data
water_all.iloc[:, 8:22].describe()
## I currently lack the domain knowledge required to make sense of most of this data.
## There are no negative numbers posted, so that's good.
## The CO3 summary is being ruined by the mass of NANs.
## A lot of the maximum values are approximately a factor of 10 greater than their respective 3rd quartiles, which leads me to suspect misplaced decimals. Further/
##/investigation required.

In [ ]:
water_all.loc[water_all['SAR'] > 15][['Na', 'CO3', 'SAR']]

In [ ]:
# Drop CO3 column and all remaining NA rows
clean_water = water_all.drop('CO3', axis = 1)
clean_water = clean_water.dropna()

clean_water.isnull().any() # NAs removed

I now consider this data to be cleaned. There might be some high values that don't make sense, but expanding my domain knowledge far enough to accurately ascertain those is beyond the scope of this project.